<a href="https://colab.research.google.com/github/thaisrezendeb/ufes_machineLearning/blob/main/Exercicio7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte III – Métodos de Classificação Baseados em Distância

**7) Realize a classificação da base de dados Car Evaluation (disponível em http://archive.ics.uci.edu/ml/) usando o kNN e Rocchio com métrica de distância de Mahalanobis. Explique os tratamentos realizados nos dados para poder usar estes classificadores. Realize 3-fold cross validation e, para cada rodada, use dois folds para a parte de calibração e um fold para teste. Na parte de calibração, o treinamento deve ser realizado usando um fold e a validação do valor de k do kNN deve ser realizado
usando o outro fold. A calibração deve ser realizada de forma a maximizar a acurácia. Expresse os resultados em forma de acurácia média, macro precisão médio, macro recall médio e tabela de contingência (de confusão) média (dado em porcentagem). Compare os resultados dos dois algoritmos e comente.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

car = pd.read_csv("car.data", sep=",", header=None)

################################################################
#Tratamento das colunas qualitativas para cálculos quantitativos
################################################################

#Primeira e segunda colunas (buying and maint)
# vhigh = 3, high = 2, med = 1, low = 0
for idx in (0,1):
  car.iloc[(car[idx] == 'vhigh'), idx] = 3
  car.iloc[(car[idx] == 'high'), idx] = 2
  car.iloc[(car[idx] == 'med'), idx] = 1
  car.iloc[(car[idx] == 'low'), idx] = 0

#Terceira coluna (doors)
# 5-more = 5
idx = 2
car.iloc[(car[idx] == '5more'), idx] = 5

#Quarta coluna (persons)
# more = 6
idx = 3
car.iloc[(car[idx] == 'more'), idx] = 6

#Quinta coluna (lug_boot)
# small = 0, med = 1, big = 2
idx = 4
car.iloc[(car[idx] == 'small'), idx] = 0
car.iloc[(car[idx] == 'med'), idx] = 1
car.iloc[(car[idx] == 'big'), idx] = 2

#Sexta coluna (safety)
# low = 0, med = 1, high = 2
idx = 5
car.iloc[(car[idx] == 'low'), idx] = 0
car.iloc[(car[idx] == 'med'), idx] = 1
car.iloc[(car[idx] == 'high'), idx] = 2

#Sétima coluna (class) 
#Não tem necessidade, alteração apenas para usar laços de repetição 
# e melhorar algoritmo
# unacc = 0, acc = 1, good = 2, vgood = 2
idx = 6
car.iloc[(car[idx] == 'unacc'), idx] = 0
car.iloc[(car[idx] == 'acc'), idx] = 1
car.iloc[(car[idx] == 'good'), idx] = 2
car.iloc[(car[idx] == 'vgood'), idx] = 3

#Converte todos os valores para inteiro
for i in range(car.shape[1]):
  car[i] = car[i].astype(int)

In [ ]:
##########################################################
#Classificação por Rocchio usando distância de Mahalanobis
##########################################################
def rocchio(car_train, car_test, conf=False, nclass=1):
  acuracia_rocchio = 0

  confusao = np.array(np.zeros((nclass, nclass))).astype(int)

  #Existem quatro classes: unacc, acc, good, vgood
  car_unacc = car_train[car_train[6] == 0]
  if car_unacc.shape[0] > 0:
    mi_unacc = car_unacc.mean()
    mi_unacc = mi_unacc[:-1]

  car_acc = car_train[car_train[6] == 1]
  if car_acc.shape[0] > 0:
    mi_acc = car_acc.mean()
    mi_acc = mi_acc[:-1]

  car_good = car_train[car_train[6] == 2]
  if car_good.shape[0] > 0:
    mi_good = car_good.mean()
    mi_good = mi_good[:-1]

  car_vgood = car_train[car_train[6] == 3]
  if car_vgood.shape[0] > 0:
    mi_vgood = car_vgood.mean()
    mi_vgood = mi_vgood[:-1]

  #Matriz de covariância para calcular distância de Mahalanobis
  roc_train = car_train.iloc[:,:-1]
  cov = roc_train.cov()
  cov_inv = np.linalg.inv(cov)

  for i, row in car_test.iterrows():
    dist_min = 1000000
    classif_new = row[6]

    rr = row.to_numpy()
    rr = np.delete(rr, 6, 0) #Exclui classificação

    #Distância de Mahalanobis para o primeiro centróide (unacc)
    if car_unacc.shape[0] > 0:
      dist = np.sqrt(np.dot(np.dot((rr-mi_unacc).T, cov_inv), (rr-mi_unacc)))      
      if dist < dist_min:
        dist_min = dist
        classif_new = 0

    #Distância de Mahalanobis para o segundo centróide (acc)
    if car_acc.shape[0] > 0:
      dist = np.sqrt(np.dot(np.dot((rr-mi_acc).T, cov_inv), (rr-mi_acc)))      
      if dist < dist_min:
        dist_min = dist
        classif_new = 1

    #Distância de Mahalanobis para o terceiro centróide (good)
    if car_good.shape[0] > 0:
      dist = np.sqrt(np.dot(np.dot((rr-mi_good).T, cov_inv), (rr-mi_good)))      
      if dist < dist_min:
        dist_min = dist
        classif_new = 2

    #Distância de Mahalanobis para o quarto centróide (vgood)
    if car_vgood.shape[0] > 0:
      dist = np.sqrt(np.dot(np.dot((rr-mi_vgood).T, cov_inv), (rr-mi_vgood)))      
      if dist < dist_min:
        dist_min = dist
        classif_new = 3

    if row[6] == classif_new:
      acuracia_rocchio = acuracia_rocchio + 1
    
    if conf == True:
      #Matriz de confusão
      for m in range(0, nclass):
        for n in range(0, nclass):
           if row[6] == m and classif_new == n:
              confusao[m][n] = confusao[m][n] + 1

  acuracia_rocchio = np.divide(acuracia_rocchio, car_test.shape[0])
  #print("Acurácia ROCCHIO:", acuracia_rocchio*100, "%")

  return acuracia_rocchio, confusao

In [ ]:
######################################################
#Classificação por kNN usando distância de Mahalanobis
######################################################
def vizinho(base_train, base_test, kviz=1, conf=False, nclass=1):
  acuracia_knn = 0
  
  confusao = np.array(np.zeros((nclass, nclass))).astype(int)
  
  if kviz > 1:
    vizinhos = np.array(np.zeros((kviz,2)), dtype=object)

  #Matriz de covariância para calcular distância de Mahalanobis
  knn_train = base_train.iloc[:,:-1]
  cov = knn_train.cov()
  cov_inv = np.linalg.inv(cov)

  #Cálculo da distância de Mahalanobis
  #Verifica a distância de cada amostra da base de teste com cada uma das amostras da base de treinamento
  for i, row in base_test.iterrows():
    dist_min = 1000000
    
    if kviz > 1:
      for n in range(0, kviz):
        vizinhos[n][1] = dist_min

    classif_new = row[base_test.shape[1]-1]

    rr = row.to_numpy()
    rr = np.delete(rr, 6, 0) #Exclui classificação
 
    for j, row_next in base_train.iterrows():
      rn = row_next.to_numpy()
      rn = np.delete(rn, 6, 0) #Exclui classificação

      dist = np.sqrt(np.dot(np.dot((rn-rr).T, cov_inv), (rn-rr)))      
      
      if kviz == 1:
        if dist < dist_min:
          dist_min = dist
          classif_new = row_next[base_test.shape[1]-1]
      else: #Mais de um vizinho, matriz de vizinhos
        for z in range(0, vizinhos.shape[0]):
          if dist < vizinhos[z][1]:
            #print("Deslocando vizinho", i, j, z, dist, vizinhos[z][1])
            #Desloca todos os vizinhos logo abaixo
            for y in range(vizinhos.shape[0]-1, z, -1):
              vizinhos[y][0] = vizinhos[y-1][0]
              vizinhos[y][1] = vizinhos[y-1][1]
            #Substitui valor menor
            vizinhos[z][1] = dist
            vizinhos[z][0] = row_next[base_test.shape[1]-1]
            break

    if kviz > 1:
      viz_df = pd.DataFrame(vizinhos, columns=['classif', 'dist'], index=None)
      classif_new = viz_df['classif'].value_counts().index[0]
      #print(classif_new)

    if row[base_test.shape[1]-1] == classif_new:
      acuracia_knn = acuracia_knn + 1

    if conf == True:
      #Matriz de confusao
      for m in range(0, nclass):
        for n in range(0, nclass):
           if row[base_test.shape[1]-1] == m and classif_new == n:
              confusao[m][n] = confusao[m][n] + 1
  
  #  print("Classif original de", i, ":", row[6], "nova classif", classif_new)
  acuracia_knn = np.divide(acuracia_knn, base_test.shape[0])

  return acuracia_knn, confusao

def idx_k(x):
  return {
      0 : 1,
      1 : 3,
      2 : 5,
      3 : 7
    }.get(x)

In [ ]:
#Divisão das amotras para 3-fold cross validation
fold = np.divide(car.shape[0],3).astype(int)

num_class = 4

#Primeira rodada
print("Primeira rodada")
print("===============")
car_train1 = car.iloc[:fold]
car_train2 = car.iloc[fold:fold*2]
car_test = car.iloc[fold*2:]

#print("Rodada 1.1")
ac_viz11_1,a = vizinho(car_train1, car_train2, kviz=1)
ac_viz11_3,a = vizinho(car_train1, car_train2, kviz=3)
ac_viz11_5,a = vizinho(car_train1, car_train2, kviz=5)
ac_viz11_7,a = vizinho(car_train1, car_train2, kviz=7)

#print("Rodada 1.2")
ac_viz12_1,a = vizinho(car_train2, car_train1, kviz=1)
ac_viz12_3,a = vizinho(car_train2, car_train1, kviz=3)
ac_viz12_5,a = vizinho(car_train2, car_train1, kviz=5)
ac_viz12_7,a = vizinho(car_train2, car_train1, kviz=7)

ac1 = (ac_viz11_1 + ac_viz12_1)/2
ac2 = (ac_viz11_3 + ac_viz12_3)/2
ac3 = (ac_viz11_5 + ac_viz12_5)/2
ac4 = (ac_viz11_7 + ac_viz12_7)/2

print("\nAcurácia NN:")
print("1-NN: {:.2f}%".format(ac1*100), \
      "\n3-NN: {:.2f}%".format(ac2*100),\
      "\n5-NN: {:.2f}%".format(ac3*100),\
      "\n7-NN: {:.2f}%".format(ac4*100))

#Faz a classificação com o k 
k = np.array([ac1, ac2, ac3, ac4]).argmax()
k = idx_k(k)

car_train = pd.concat([car_train1, car_train2])

ac_rocch1, conf_r1 = rocchio(car_train, car_test, conf=True, nclass=num_class)
print("\nAcurácia Rocchio: {:.2f}%".format(ac_rocch1*100))

ac_viz1, conf1 = vizinho(car_train, car_test, k, conf=True, nclass=num_class)
print("Acurácia para k = {}: {:.2f}%".format(k, ac_viz1*100))

#Segunda rodada
print("\n\nSegunda rodada")
print("==============")
car_train1 = car.iloc[:fold]
car_test = car.iloc[fold:fold*2]
car_train2 = car.iloc[fold*2:]

#print("Rodada 2.1")
ac_viz21_1,a = vizinho(car_train1, car_train2, kviz=1)
ac_viz21_3,a = vizinho(car_train1, car_train2, kviz=3)
ac_viz21_5,a = vizinho(car_train1, car_train2, kviz=5)
ac_viz21_7,a = vizinho(car_train1, car_train2, kviz=7)

#print("Rodada 2.2")
ac_viz22_1,a = vizinho(car_train2, car_train1, kviz=1)
ac_viz22_3,a = vizinho(car_train2, car_train1, kviz=3)
ac_viz22_5,a = vizinho(car_train2, car_train1, kviz=5)
ac_viz22_7,a = vizinho(car_train2, car_train1, kviz=7)

ac1 = (ac_viz21_1 + ac_viz22_1)/2
ac2 = (ac_viz21_3 + ac_viz22_3)/2
ac3 = (ac_viz21_5 + ac_viz22_5)/2
ac4 = (ac_viz21_7 + ac_viz22_7)/2

print("\nAcurácia NN:")
print("1-NN: {:.2f}%".format(ac1*100),\
      "\n3-NN: {:.2f}%".format(ac2*100),\
      "\n5-NN: {:.2f}%".format(ac3*100),\
      "\n7-NN: {:.2f}%".format(ac4*100))

#Faz a classificação com o k 
k = np.array([ac1, ac2, ac3, ac4]).argmax()
k = idx_k(k)

car_train = pd.concat([car_train1, car_train2])

ac_rocch2, conf_r2 = rocchio(car_train, car_test, conf=True, nclass=num_class)
print("\nAcurácia Rocchio: {:.2f}%".format(ac_rocch2*100))

ac_viz2, conf2 = vizinho(car_train, car_test, k, conf=True, nclass=num_class)
print("Acurácia para k = {}: {:.2f}%".format(k, ac_viz2*100))

#Terceira rodada
print("\n\nTerceira rodada")
print("===============")
car_test = car.iloc[:fold]
car_train1 = car.iloc[fold:fold*2]
car_train2 = car.iloc[fold*2:]

#print("Rodada 3.1")
ac_viz31_1,a = vizinho(car_train1, car_train2, kviz=1)
ac_viz31_3,a = vizinho(car_train1, car_train2, kviz=3)
ac_viz31_5,a = vizinho(car_train1, car_train2, kviz=5)
ac_viz31_7,a = vizinho(car_train1, car_train2, kviz=7)

#print("Rodada 3.2")
ac_viz32_1,a = vizinho(car_train2, car_train1, kviz=1)
ac_viz32_3,a = vizinho(car_train2, car_train1, kviz=3)
ac_viz32_5,a = vizinho(car_train2, car_train1, kviz=5)
ac_viz32_7,a = vizinho(car_train2, car_train1, kviz=7)

ac1 = (ac_viz31_1 + ac_viz32_1)/2
ac2 = (ac_viz31_3 + ac_viz32_3)/2
ac3 = (ac_viz31_5 + ac_viz32_5)/2
ac4 = (ac_viz31_7 + ac_viz32_7)/2

print("\nAcurácia NN:")
print("1-NN: {:.2f}%".format(ac1*100),\
      "\n3-NN: {:.2f}%".format(ac2*100),\
      "\n5-NN: {:.2f}%".format(ac3*100),\
      "\n7-NN: {:.2f}%".format(ac4*100))

#Faz a classificação com o k 
k = np.array([ac1, ac2, ac3, ac4]).argmax()
k = idx_k(k)

car_train = pd.concat([car_train1, car_train2])

ac_rocch3, conf_r3 = rocchio(car_train, car_test, conf=True, nclass=num_class)
print("\nAcurácia Rocchio: {:.2f}%".format(ac_rocch3*100))

ac_viz3, conf3 = vizinho(car_train, car_test, k, conf=True, nclass=num_class)
print("Acurácia para k = {}: {:.2f}%".format(k, ac_viz3*100))

Primeira rodada

Acurácia NN:
1-NN: 78.30% 
3-NN: 76.65% 
5-NN: 77.78% 
7-NN: 76.91%

Acurácia Rocchio: 30.56%
Acurácia para k = 1: 81.42%


Segunda rodada

Acurácia NN:
1-NN: 62.07% 
3-NN: 62.41% 
5-NN: 61.81% 
7-NN: 61.46%

Acurácia Rocchio: 79.17%
Acurácia para k = 3: 81.77%


Terceira rodada

Acurácia NN:
1-NN: 81.42% 
3-NN: 79.69% 
5-NN: 79.51% 
7-NN: 77.78%

Acurácia Rocchio: 52.95%
Acurácia para k = 1: 79.51%


In [ ]:
#Acurácia média Rocchio
print("Acurácia média Rocchio: {:.2f}%".format((ac_rocch1+ac_rocch2+ac_rocch3)/3*100))

#Acurácia média kNN
print("Acurácia média kNN: {:.2f}%".format((ac_viz1+ac_viz2+ac_viz3)/3*100))

#Cálculo da macro precisão para kNN
#np.set_printoptions(precision=2)
confusao_total = conf1 + conf2 + conf3
confusao_perc = np.divide(confusao_total, car.shape[0])
confusao_perc = np.multiply(confusao_perc, 100)
confusao_perc = pd.DataFrame(confusao_perc, \
                           index=['unacc', 'acc', 'good', 'vgood'], \
                           columns=['unacc', 'acc', 'good', 'vgood'])
print("\nMatriz de confusão kNN (%):")
print(confusao_perc)

#FP soma a linha
p = 0
for i in range(0,num_class):
  if confusao_total.sum(axis=1)[i] != 0:
    p = p + (confusao_total[i][i] / confusao_total.sum(axis=1)[i])

p = p / num_class
print("\nMacro precisão kNN: {:.2f}%".format(p*100))

#Cálculo da macro revocação para kNN
#FN soma a coluna
r = 0
for i in range(0,num_class):
  if confusao_total.sum(axis=0)[i] != 0:
    r =  r + (confusao_total[i][i] / confusao_total.sum(axis=0)[i])
        
r = r / num_class
print("Macro revocação kNN: {:.2f}%".format(r*100))

#Cálculo da macro precisão para Rocchio
confusao_rocchio = conf_r1 + conf_r2 + conf_r3
confusao_pr = np.divide(confusao_rocchio, car.shape[0])
confusao_pr = np.multiply(confusao_pr, 100)
confusao_pr = pd.DataFrame(confusao_pr, \
                           index=['unacc', 'acc', 'good', 'vgood'], \
                           columns=['unacc', 'acc', 'good', 'vgood'])
print("\nMatriz de confusão Rocchio (%):")
print(confusao_pr)

pr = 0
for i in range(0,num_class):
  if confusao_rocchio.sum(axis=1)[i] != 0:
    pr = pr + (confusao_rocchio[i][i] / confusao_rocchio.sum(axis=1)[i])
        
pr = pr / num_class
print("\nMacro precisão Rocchio: {:.2f}%".format(pr*100))

#Cálculo da macro revogação para Rocchio
rr = 0
for i in range(0,num_class):
  if confusao_rocchio.sum(axis=0)[i] != 0:
    rr = rr + (confusao_rocchio[i][i] / confusao_rocchio.sum(axis=0)[i])
      
rr = rr / num_class
print("Macro revogação Rocchio: {:.2f}%".format(rr*100))

Acurácia média Rocchio: 54.22%
Acurácia média kNN: 80.90%

Matriz de confusão kNN (%):
           unacc        acc      good     vgood
unacc  62.789352   7.233796  0.000000  0.000000
acc     2.546296  15.682870  1.851852  2.141204
good    0.000000   3.993056  0.000000  0.000000
vgood   0.000000   1.331019  0.000000  2.430556

Macro precisão kNN: 56.21%
Macro revocação kNN: 51.20%

Matriz de confusão Rocchio (%):
           unacc        acc      good      vgood
unacc  37.615741  19.444444  0.231481  12.731481
acc     1.793981  13.078704  0.289352   7.060185
good    0.000000   0.289352  0.000000   3.703704
vgood   0.000000   0.231481  0.000000   3.530093

Macro precisão Rocchio: 51.60%
Macro revogação Rocchio: 37.02%


A acurácia média obtida na classificação por Rocchio usando a distância de Mahalanobis foi de **54,22%**, enquanto a acurácia média por kNN, também usando a distância de Mahalanobis, foi de **80,90%**. Tanto a macro precisão quanto a macro revogação do kNN foram superiores aos resultados obtidos por Rocchio: **56,21%** e **51,20%** contra **51,60%** e **37,02%**, respectivamente. A matriz de confusão mostra que o kNN teve mais acertos do que o Rocchio (analisando a diagonal principal). Uma observação importante é que não houve acertos para a classe **good** em nenhum dos dois métodos.